In [ ]:
import pandas as pd
import geonamescache
import unidecode
import re
import random

In [97]:
f = open("data/headlines.txt")
headlines = f.readlines()
f.close()

In [98]:
headlines = [unidecode.unidecode(h.strip()) for h in headlines]

In [38]:
Headlines = pd.DataFrame(headlines)

In [39]:
Headlines.columns= ["headline"]
Headlines['city'] = ''
Headlines['country'] = ''

In [40]:
gc = geonamescache.GeonamesCache()

In [41]:
countries = gc.get_countries()

In [42]:
cities = gc.get_cities()

In [43]:
Countries= pd.DataFrame([(cc,unidecode.unidecode(countries[cc]["name"])) for cc in countries])

In [44]:
Countries.columns=("countryCode","country")

In [45]:
Countries.head()

,countryCode,country
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla


In [46]:
Cities = pd.DataFrame([(c,unidecode.unidecode(cities[c]["name"]), cities[c]["countrycode"], cities[c]['latitude'], cities[c]['longitude']) for c in cities])

In [47]:
Cities.columns = ("id", "city", "countryCode","latitude","longitude")

In [48]:
Cities = pd.merge(Cities,Countries, on='countryCode',how='left')

In [49]:
Cities.head()

,id,city,countryCode,latitude,longitude,country
0,3041563,Andorra la Vella,AD,42.50779,1.52109,Andorra
1,290594,Umm Al Quwain City,AE,25.56473,55.55517,United Arab Emirates
2,291074,Ras Al Khaimah City,AE,25.78953,55.94320,United Arab Emirates
3,291580,Zayed City,AE,23.65416,53.70522,United Arab Emirates
4,291696,Khawr Fakkan,AE,25.33132,56.34199,United Arab Emirates


In [50]:
for cc in Countries.country:
    for i,h in Headlines[Headlines.headline.str.contains(r'\b'+cc+'\\b')].iterrows():
        Headlines.iloc[i]['country'] = cc

In [51]:
Headlines[Headlines['country'] != ""].head()

,headline,city,country
3,"Mystery Virus Spreads in Recife, Brazil",,Brazil
25,Zika cases in Vietnam's Ho Chi Minh City surge,,Vietnam
30,Thailand-Zika Virus in Bangkok,,Thailand
44,"Zika outbreak in Piracicaba, Brazil",,Brazil
58,"Zika surfaces in Klang, Malaysia",,Malaysia


In [52]:
for c in Cities.city:
    for i,h in Headlines[Headlines.headline.str.contains(r'\b'+c+'\\b')].iterrows():
        Headlines.iloc[i]['city'] = c

C:\Users\pborg\AppData\Local\Continuum\anaconda3\envs\discovering-disease-outbreaks\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [100]:
Headlines.head()

,headline,city,country
0,Zika Outbreak Hits Miami,Miami,
1,Could Zika Reach New York City?,New York City,
2,First Case of Zika in Miami Beach,Miami Beach,
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,


In [54]:
Cities.head()

,id,city,countryCode,latitude,longitude,country
0,3041563,Andorra la Vella,AD,42.50779,1.52109,Andorra
1,290594,Umm Al Quwain City,AE,25.56473,55.55517,United Arab Emirates
2,291074,Ras Al Khaimah City,AE,25.78953,55.94320,United Arab Emirates
3,291580,Zayed City,AE,23.65416,53.70522,United Arab Emirates
4,291696,Khawr Fakkan,AE,25.33132,56.34199,United Arab Emirates


In [55]:
Countries.head()

,countryCode,country
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla


In [121]:
H1 = pd.merge(Headlines[Headlines['country'] != ""],Countries, on='country', how='left')

In [122]:
H1.head()

,headline,city,country,countryCode
0,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,BR
1,Zika cases in Vietnam's Ho Chi Minh City surge,Ho Chi Minh City,Vietnam,VN
2,Thailand-Zika Virus in Bangkok,Bangkok,Thailand,TH
3,"Zika outbreak in Piracicaba, Brazil",Piracicaba,Brazil,BR
4,"Zika surfaces in Klang, Malaysia",Klang,Malaysia,MY


In [123]:
H1 = pd.merge(H1,Cities,on=['countryCode','country','city'], how='left')

In [124]:
H1.head()

,headline,city,country,countryCode,id,latitude,longitude
0,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,BR,3390760,-8.05389,-34.88111
1,Zika cases in Vietnam's Ho Chi Minh City surge,Ho Chi Minh City,Vietnam,VN,1566083,10.82302,106.62965
2,Thailand-Zika Virus in Bangkok,Bangkok,Thailand,TH,1609350,13.75398,100.50144
3,"Zika outbreak in Piracicaba, Brazil",Piracicaba,Brazil,BR,3453643,-22.72528,-47.64917
4,"Zika surfaces in Klang, Malaysia",Klang,Malaysia,MY,1732905,3.03667,101.44333


In [125]:
H2 = pd.merge(Headlines[Headlines['country'] == ""],Cities, on='city', how='left')

In [126]:
H2.head()

,headline,city,country_x,id,countryCode,latitude,longitude,country_y
0,Zika Outbreak Hits Miami,Miami,,4164138,US,25.77427,-80.19366,United States
1,Could Zika Reach New York City?,New York City,,5128581,US,40.71427,-74.00597,United States
2,First Case of Zika in Miami Beach,Miami Beach,,4164143,US,25.79065,-80.13005,United States
3,Dallas man comes down with case of Zika,Dallas,,4684888,US,32.78306,-96.80667,United States
4,Dallas man comes down with case of Zika,Dallas,,5722064,US,44.91928,-123.31705,United States


In [127]:
H2.drop_duplicates(subset=['headline','city'],inplace=True)

In [128]:
H1.drop(['id'], axis=1,inplace=True)
H2.drop(['id'], axis=1,inplace=True)
H2.drop('country_x', axis=1,inplace=True)
H2.rename(columns={'country_y':'country'},inplace=True)

In [129]:
H = H1.append(H2,sort=False)

In [130]:
[H1.shape,H2.shape,H.shape]

[(16, 6), (633, 6), (649, 6)]

In [131]:
H[H['country'].isnull()].head(10)

,headline,city,country,countryCode,latitude,longitude
26,Louisiana Zika cases up to 26,,NaN,NaN,NaN,NaN
28,Zika infects pregnant woman in Cebu,,NaN,NaN,NaN,NaN
71,Spanish Flu Sighted in Antigua,,NaN,NaN,NaN,NaN
89,Carnival under threat in Rio De Janeiro due to...,,NaN,NaN,NaN,NaN
104,Zika case reported in Oton,,NaN,NaN,NaN,NaN
107,Hillsborough uses innovative trap against Zika...,,NaN,NaN,NaN,NaN
121,Maka City Experiences Influenza Outbreak,,NaN,NaN,NaN,NaN
199,More Zika patients reported in Mcallen,,NaN,NaN,NaN,NaN
230,West Nile Virus Outbreak in Saint Johns,,NaN,NaN,NaN,NaN
355,More people in Mclean are infected with Hepati...,,NaN,NaN,NaN,NaN


In [133]:
H.groupby(['city']).count().reset_index().sort_values('headline', ascending=False).head(10)

,city,headline,country,countryCode,latitude,longitude
0,,42,0,0,0,0
331,Monroe,4,4,4,4,4
320,Miami,4,4,4,4,4
478,Springs,4,4,4,4,4
43,Bedford,3,3,3,3,3
296,Madrid,3,3,3,3,3
35,Barcelona,3,3,3,3,3
438,San Jose,2,2,2,2,2
234,Jefferson,2,2,2,2,2
59,Birmingham,2,2,2,2,2


SAVE the data

In [139]:
H.reset_index().to_json('./data/headlines_city_country_coords')